[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ACS-IC-labs/IC-labs/blob/main/teme/tema25/tema_2025.ipynb)

# **Tema Criptologie 2025**

## Task 1 - Break the LFSR (30p)


Am discutat la curs și laborator despre cum securitatea unui cifru șir (stream cipher) este influențată de securitatea PRG-ului folosit pentru a genera cheia. În acest exercițiu, vom vedea cum putem ataca un cifru șir bazat pe un LFSR ([Linear-Feedback Shift Register](https://en.wikipedia.org/wiki/Linear-feedback_shift_register)) liniar.

Natura cifrului șir bazat pe LFSR îl face vulnerabil la un atac simplu de tipul **known-plaintext attack**. LFSR a fost utilizat o perioada îndelungată în principal în dispozitivele hardware datorită simplității de implementare, dar de multe ori, componentele nu mai puteau fi modificate după producție. Prin urmare, numărul de biți ai sistemului criptografic și definiția care determină modul de generare al biților următori ar trebui să fie considerate neschimbabile și informație publică, iar accesul fizic asupra dispozitivului ar permite unui atacator să determine aceste valori. Prin urmare, numai valoarea de inițializare ar trebui considerată secretă și parte a cheii de criptare, conform [principiilor lui Kerckhoff](https://en.wikipedia.org/wiki/Kerckhoffs%27s_principle).

> **Info**
>
> Acesta a fost cazul mașinii Enigma folosite în cel de-al Doilea Război Mondial. Cablajul rotorului a fost același în fiecare mașină și nu mai putea fi schimbat odată ce a fost produs. Prin urmare, ar trebui să presupuneți că atacatorul cunoaște acele configurații de cablare, deoarece este nevoie de un singur dispozitiv capturat pentru ca setările să fie determinate.

Ghicind doar o parte din textul în clar (plaintext) și având acces la întregul text cifrat (ciphertext), ne permite să deducem suficientă informație pentru a recrea întreaga cheie și, astfel, să descifrăm restul mesajului.

Mai jos oferim funcțiile ajutătoare pentru conversii de date și operația xor, disponibile din cadrul laboratoarelor.

In [ ]:
import base64
from typing import Generator


def _pad(data: str, size: int) -> str:
    reminder = len(data) % size
    if reminder != 0:
        data = "0" * (size - reminder) + data
    return data


def _chunks(data: str, chunk_size: int) -> Generator[str, None, None]:
    data = _pad(data, chunk_size)
    for i in range(0, len(data), chunk_size):
        yield data[i : i + chunk_size]


def _hex(data: int) -> str:
    return format(data, "02x")


# Conversion functions


def hex_2_bin(data: str) -> str:
    """Converts a hexadecimal string to a binary representation.

    Args:
        data (str): The hexadecimal string to be converted. It should have an
            even number of characters and only contain valid hexadecimal digits
            (0-9, A-F, a-f).

    Returns:
        str: The binary representation of the hexadecimal string, where each
            pair of hexadecimal digits is encoded as an 8-bit binary number.

    Examples:
        >>> hex_2_bin("01abcd")
        '000000011010101111001101'
        >>> hex_2_bin("0a")
        '00001010'
    """
    return "".join(f"{int(x, 16):08b}" for x in _chunks(data, 2))


def bin_2_hex(data: str) -> str:
    """Converts a binary string to a hexadecimal representation.

    Args:
        data (str): The binary string to be converted. It should have a multiple
            of 8 characters and only contain valid binary digits (0 or 1).

    Returns:
        str: The hexadecimal representation of the binary string, where each
            group of 8 binary digits is encoded as a pair of hexadecimal digits.

    Examples:
        >>> bin_2_hex("000000011010101111001101")
        '01abcd'
        >>> bin_2_hex("00001010")
        '0a'
    """
    return "".join(f"{int(b, 2):02x}" for b in _chunks(data, 8))


def str_2_bin(data: str) -> str:
    """Converts a string to a binary representation.

    Args:
        data (str): The string to be converted.

    Returns:
        str: The binary representation of the string, where each character is
            encoded as an 8-bit binary number.

    Examples:
        >>> str_2_bin("Hello")
        '0100100001100101011011000110110001101111'
        >>> str_2_bin("IC")
        '0100100101000011'
    """
    return "".join(f"{ord(c):08b}" for c in data)


def bin_2_str(data: str) -> str:
    """Converts a binary string to a string.

    Args:
        data (str): The binary string to be converted. It should have a multiple
            of 8 characters and only contain valid binary digits (0 or 1).

    Returns:
        str: The string representation of the binary string, where each group
            of 8 binary digits is decoded as a character.

    Examples:
        >>> bin_2_str("0100100001100101011011000110110001101111")
        'Hello'
        >>> bin_2_str("0100100101000011")
        'IC'
    """
    return "".join(chr(int(b, 2)) for b in _chunks(data, 8))


def str_2_hex(data: str) -> str:
    """Converts a string to a hexadecimal representation.

    Args:
        data (str): The string to be converted.

    Returns:
        str: The hexadecimal representation of the string, where each character
            is encoded as a pair of hexadecimal digits.

    Examples:
        >>> str_2_hex("Hello")
        '48656c6c6f'
        >>> str_2_hex("IC")
        '4943'
    """
    return "".join(f"{ord(c):02x}" for c in data)


def hex_2_str(data: str) -> str:
    """Converts a hexadecimal string to a string.

    Args:
        data (str): The hexadecimal string to be converted. It should have an
            even number of characters and only contain valid hexadecimal digits
            (0-9, A-F, a-f).

    Returns:
        str: The string representation of the hexadecimal string, where each
            pair of hexadecimal digits is decoded as a character.

    Examples:
        >>> hex_2_str("48656c6c6f")
        'Hello'
        >>> hex_2_str("4943")
        'IC'
    """
    return "".join(chr(int(x, 16)) for x in _chunks(data, 2))


# XOR functions


def strxor(operand_1: str, operand_2: str) -> str:
    """Performs a bitwise exclusive OR (XOR) operation on two strings.

    Args:
        operand_1 (str): The first string to be XORed.
        operand_2 (str): The second string to be XORed.

    Returns:
        str: The result of the XOR operation on the two strings, where each
            character is encoded as an 8-bit binary number. The result has
            the same length as the shorter input string.

    Examples:
        >>> strxor("Hello", "IC")
        '\\x01&'
        >>> strxor("secret", "key")
        '\\x18\\x00\\x1a'
    """
    return "".join(chr(ord(x) ^ ord(y)) for (x, y) in zip(operand_1, operand_2))


def bitxor(operand_1: str, operand_2: str) -> str:
    """Performs a bitwise exclusive OR (XOR) operation on two bit-strings.

    Args:
        operand_1 (str): The first bit-string to be XORed. It should only
            contain valid binary digits (0 or 1).
        operand_2 (str): The second bit-string to be XORed. It should only
            contain valid binary digits (0 or 1).

    Returns:
        str: The result of the XOR operation on the two bit-strings, where each
            bit is encoded as a character. The result has the same length as
            the shorter input bit-string.

    Examples:
        >>> bitxor("01001000", "01000010")
        '00001010'
        >>> bitxor("10101010", "00110011")
        '10011001'
    """
    return "".join(str(int(x) ^ int(y)) for (x, y) in zip(operand_1, operand_2))


def hexxor(operand_1: str, operand_2: str) -> str:
    """Performs a bitwise exclusive OR (XOR) operation on two hexadecimal
    strings.

    Args:
        operand_1 (str): The first hexadecimal string to be XORed. It should
            have an even number of characters and only contain valid hexadecimal
            digits (0-9, A-F, a-f).
        operand_2 (str): The second hexadecimal string to be XORed. It should
            have an even number of characters and only contain valid
            digits (0-9, A-F, a-f).

    Returns:
        str: The result of the XOR operation on the two hexadecimal strings,
            where each pair of hexadecimal digits is encoded as a pair of
            hexadecimal digits. The result has the same length as the shorter
            input hexadecimal string.

    Examples:
        >>> hexxor("48656c6c6f", "42696e67")
        '0a0c020b'
        >>> hexxor("736563726574", "6b6579")
        '18001a'
    """
    return "".join(
        _hex(int(x, 16) ^ int(y, 16))
        for (x, y) in zip(_chunks(operand_1, 2), _chunks(operand_2, 2))
    )


# Python3 'bytes' functions


def bytes_to_string(bytes_data: bytearray | bytes) -> str:
    """Converts a byte array or a byte string to a string.

    Args:
        bytes_data (bytearray | bytes): The byte array or the byte string to be
            converted. It should be encoded in Latin-1 format.

    Returns:
        str: The string representation of the byte array or the byte string,
            decoded using Latin-1 encoding.

    Examples:
        >>> bytes_to_string(b'Hello')
        'Hello'
        >>> bytes_to_string(bytearray(b'IC'))
        'IC'
    """
    return bytes_data.decode(encoding="raw_unicode_escape")


def string_to_bytes(string_data: str) -> bytes:
    """Converts a string to a byte string.

    Args:
        string_data (str): The string to be converted.

    Returns:
        bytes: The byte string representation of the string, encoded using
        Latin-1 encoding.

    Examples:
        >>> string_to_bytes('Hello')
        b'Hello'
        >>> string_to_bytes('IC')
        b'IC'
    """
    return string_data.encode(encoding="raw_unicode_escape")


# Base64 functions


def b64encode(data: str) -> str:
    """Encodes a string to base64.

    Parameters:
        data (str): The string to be encoded.

    Returns:
        str: The base64 encoded string, using Latin-1 encoding.

    Examples:
        >>> b64encode("Hello")
        'SGVsbG8='
        >>> b64encode("IC")
        'SUM='
    """
    return bytes_to_string(base64.b64encode(string_to_bytes(data)))


def b64decode(data: str) -> str:
    """Decodes a base64 encoded string.

    Args:
        data (str): The base64 encoded string to be decoded. It should only
            contain valid base64 characters (A-Z, a-z, 0-9, +, /, =).

    Returns:
        str: The decoded string, using Latin-1 encoding.

    Examples:
        >>> b64decode("SGVsbG8=")
        'Hello'
        >>> b64decode("SUM=")
        'IC'
    """
    return bytes_to_string(base64.b64decode(string_to_bytes(data)))

În continuare, vom folosi urmatoarea implementare de LFSR, care depinde de pozițiile care determină starea urmatoare, numite _taps_, și _starea inițială_ a PRG-ului. Valorile _taps_ au fost alese așa fel încât perioada maximă pe care o poate genera LFS-ul este $2^n-1$, unde $n$ reprezintă numărul de biți din stare. În cazul nostru, LFSR-ul are o stare de 88 bits și implementează taps `[88, 87, 17, 16]`.

> **Info**
>
> Pentru mai multe detalii legate de alegerea parametrilor LFSR, puteți consulta documentația de la [Xilinx](https://docs.amd.com/v/u/en-US/xapp052) și acest [blog](https://www.moria.us/articles/demystifying-the-lfsr/).

Mesajul în clar care a fost criptat este de forma `CRYPTO_CTF{<text>}` (`<text>` este un placeholder pentru conținut). Un atac de tip brute-force nu este practic în condițiile acestui LFSR. Task-ul vostru este să implementați un atac pornind de la informația pe care o știți despre mesajul în clar și încercați să determinați starea internă a PRG-ului, urmând ca apoi să generați restul secvenței de biți și să decriptați flagul.

> **Hint**
>
> Ce reprezintă **primii $n$ biți** returnați de LFSR?

> **Notă**
>
> Veți ști că atacul a decurs cu succes atunci când veți putea citi textul flagului.

In [ ]:
from functools import reduce

class LFSR:
    def __init__(self, taps: list[int], seed: str) -> None:
        """
        Initializes a Linear Feedback Shift Register (LFSR).

        Args:
            taps (list[int]): A list of bit positions (1-indexed) used in the feedback function.
                These positions determine which bits will be XORed together to generate the next bit.
            seed (str): The initial state of the LFSR as a string of '0' and '1' characters.
                The length of this string determines the size of the LFSR.

        Examples:
            >>> lfsr = LFSR([3, 5], "01101")  # Creates a 5-bit LFSR with taps at positions 3 and 5
            >>> lfsr = LFSR([1, 2], "111")    # Creates a 3-bit LFSR with taps at positions 1 and 2
        """
        self.taps: list[int] = taps
        self.state: str = seed

    def next_bit(self) -> str:
        """
        Generates the next bit in the LFSR sequence.

        Returns:
            str: The next bit in the sequence, which can be either '0' or '1'.
        """
        bits = [self.state[t - 1] for t in self.taps]
        new_bit = reduce(bitxor, bits)
        result = self.state[-1]
        self.state = new_bit + self.state[0:-1]
        return result

    def generate(self, num_bits: int) -> str:
        """
        Generates a sequence of bits using the LFSR.

        Args:
            num_bits (int): The number of bits to generate.

        Returns:
            str: A string representing the generated sequence of bits.
        """
        return "".join([self.next_bit() for _ in range(num_bits)])

Flag-ul a fost criptat folosind codul de mai jos.

> **Info**
>
> Dacă doriți să reproduceți local task-ul, de exemplu, pentru debugging, puteți copia scriptul de mai jos într-o celula de cod și executa. Va trebui să creați înainte un fisier `flag.txt` care să conțină flagul sub forma `CRYPTO_CTF{<text>}`.

```python
import os

with open("flag.txt", "r") as f:
    flag_bin = str_2_bin(f.read().strip())

taps = [88, 87, 17, 16]
seed = os.urandom(11).hex()
lfsr = LFSR(taps, hex_2_bin(seed))

key = lfsr.generate(len(flag_bin))

flag_enc = bin_2_hex(bitxor(flag_bin, key))
with open("flag.enc", "w") as f:
    _ = f.write(flag_enc)
```

Descărcați fișierul cu flagul, rulând celula de mai jos.

In [ ]:
![ -f flag.enc ] && echo "File already exists." || wget https://github.com/ACS-IC-labs/IC-labs/raw/main/teme/tema25/flag.enc

Completați codul următor pentru a efectua atacul. Citirea flagului criptat este deja realizată.

In [ ]:
with open("flag.enc", "r") as f:
    flag_enc = f.read().strip()

########## SOLUȚIA AICI ##########

cipher_bin = hex_2_bin(flag_enc)

# Known part of the plaintext
known_plaintext = "CRYPTO_CTF{"
known_plain_bin = str_2_bin(known_plaintext)

# Extract keystream
keystream_start = bitxor(known_plain_bin, cipher_bin[:len(known_plain_bin)])

# Important: reverse the 88-bit keystream segment before using as seed
seed = keystream_start[:88][::-1]

# Initialize LFSR and generate keystream
lfsr = LFSR(taps, seed)
full_keystream = lfsr.generate(len(cipher_bin))

# Decrypt
flag_bin = bitxor(cipher_bin, full_keystream)
flag = bin_2_str(flag_bin)
##################################

print(flag)

## Task 2 - Differential Cryptanalysis (70p)

Avem următorul program de gestionare a notițelor. Programul ne permite să scriem notițe și să citim notițele scrise. Desigur, mesajele salvate vor trebui criptate, pentru a ne asigura că nu pot fi citite de altcineva.

Algoritmul este unul relativ simplu, bazat pe cifrul bloc [Nimbus](https://en.wikipedia.org/wiki/Nimbus_(cipher)). Cu o cheie de 128 bits și blocuri de 64 bits, algoritmul criptează mesajul în 5 runde, așa cum se poate observa în funcția `encrypt_block()`. Rundele pot fi descrise prin $Y_i = K_i^{odd} \cdot g(Y_{i-1} \oplus K_i)$, unde $K_i$ și $K_i^{odd}$ sunt subcheile rundei, iar  $K_i^{odd}$ este o subcheie forțată să fie impară.
De asemenea, funcția $g(\cdot)$ inversează biții unui bloc, ceea ce oferă algoritmului și o proprietate de *neliniaritate*.

*Asigurați-vă că ați înțeles funcția `encrypt_block()` înainte de a implementa atacul.*

In [ ]:
import binascii

ROUNDS = 5
BLOCK_LEN = 8
HEX_BLOCK_LEN = BLOCK_LEN * 2
MAX_NOTES = 2048
MAX_NOTE_LEN = 512


def mod_64(val: int) -> int:
    """Calculează val modulo 2**64"""
    return val & ((1 << 64) - 1)


def pad(plaintext: bytes) -> bytes:
    if len(plaintext) % BLOCK_LEN != 0:
        return plaintext + b"\0" * (BLOCK_LEN - (len(plaintext) % BLOCK_LEN))
    return plaintext


def g(x: int) -> int:
    b = format(x, "b").rjust(BLOCK_LEN * 8, "0")
    return int(b[::-1], 2)


def encrypt_block(plaintext: bytes) -> str:
    """Encrypts a given block of plaintext"""
    k = b"BARWackCcZimOaioafgvZlRrneCYxTHbXSnaqDpY"

    assert len(plaintext) * ROUNDS == len(k)

    result = int.from_bytes(plaintext, byteorder="big")
    for i in range(ROUNDS):
        # Fiecare pas al for-ului reprezintă o rundă de criptare
        # Obținem un block din cheia principală pentru a-l folosi ca subcheie
        key = int.from_bytes(
            k[i * BLOCK_LEN : (i + 1) * BLOCK_LEN],
            byteorder="big",
        )

        # Ne asigurăm ca vom avea o cheie de valoare impară
        key_odd = key | 1

        # XOR-am blocul curent cu cheia
        result ^= key

        # Efectuăm un inversarea biților
        result = g(result)

        # Efectuăm înmulțirea cu cheia impară și ne asigurăm că rămânem cu un
        # block de dimensiune 64 biti
        result = mod_64(result * key_odd)

    # Transformăm rezultatul în format hex
    return format(result, "x").rjust(HEX_BLOCK_LEN, "0")


def encrypt(msg: bytes) -> str:
    plain = pad(msg)
    result = ""
    for i in range(0, len(plain), BLOCK_LEN):
        result += encrypt_block(plain[i : i + BLOCK_LEN])
    return result


def menu(notes: list[str]) -> list[str] | None:
    print(
        """
1) Store
2) Retrieve
Anything else to quit
"""
    )
    ui = input("Selection? ").rstrip()
    if ui == "1":
        if len(notes) >= MAX_NOTES:
            print("Max capacity")
            return notes
        ui = input("Give me a note to encrypt: ").rstrip()
        try:
            msg = binascii.unhexlify(ui)
        except Exception as e:
            print("Invalid input.", e)
            return notes
        if len(msg) > MAX_NOTE_LEN:
            print("Invalid input.")
        else:
            notes.append(encrypt(msg))
    elif ui == "2":
        ui = input("What index would you like to read? ").rstrip()
        try:
            index = int(ui)
        except:
            print("Invalid index.")
            return notes
        if index >= 0 and index < len(notes):
            print(notes[index])
        else:
            print("Invalid index.")
    else:
        return None
    return notes


# Exemplu de rulare meniu interactiv:
#
# notes = []
# while notes is not None:
# 	notes = menu(notes)

Atacul pe care ne dorim să îl implementăm este descris în [Differential Cryptanalysis of Nimbus](https://link.springer.com/content/pdf/10.1007/3-540-45473-X_16.pdf), pentru a experimenta cu conceptul de Differential Cryptanalysis pe un algoritm de criptare mai simplu decât full DES. Așa cum este descris și în articol, putem afla cheia $K$ cu doar 256 plaintexts și testând doar 1024 subchei posibile. În această temă puteți încerca să implementați întocmai atacul descris în articol sau puteți folosi parțial ideea. Va trebui să urmăm pașii unui atac clasic de Differential Cryptanalysis.

Pentru a vă familiariza cu ideea de Differential Cryptanalysis, puteți citi și explicațiile din acest [link](http://theamazingking.com/crypto-feal.php).

Pașii pe care va trebui să îi urmăm sunt:

1. Să descoperim un diferențial Δ
2. Folosind diferențialul Δ, analizăm ce se întâmplă pe o porțiune a algoritmului de criptare
3. Identificăm toate modificările prin algoritm și descoperim astfel subcheile

Pasul 1 presupune să descoperim o caracteristică care se va păstra la fiecare rundă. În cazul algoritmului de criptare Nimbus, funcția $g(\cdot)$ este cea care prezintă o astfel de vulnerabilitate. Vom alege $Δ = 2^{63} - 2$ (puteți verifica analiza detaliată din articolul care descrie atacul).

Având diferențialul Δ calculat, vom avea nevoie de 256 plaintexts pentru a realiza atacul. Completați funcția de mai jos astfel încât să generați 128 plaintext-uri random, alături de perechile lor pentru care p XOR p' = Δ.

Cu $Δ = 2^{63} - 2$ și folosindu-ne de ideea subcheilor obligatorii impare, putem sa obținem perechi de ciphertext $(C_{1}, C_{2})$.

![Formula](https://i.ibb.co/bJSGzQR/Formula.png)

Pentru a rezolva o astfel de ecuație în modulo, vom avea de rezolvat o congruență liniară ([laboratorul 3](https://ocw.cs.pub.ro/courses/ic/labs/03)) de forma:

$$a x \equiv b \pmod{n} \iff a x - n y = b$$

Soluțiile se pot afla folosind algoritmul lui Euclid extins, implementare pe care o aveți deja în schelet.

Vă rămâne de implementat aflarea subcheilor, folosind ciphertext-urile generate anterior (detalii în cod).

În final, veți avea de implementat recursiv atacul pe fiecare rundă a algoritmului, reconstituind toate drumurile plauzibile pe care le poate avea un ciphertext, rămânând cu subcheile potrivite. Apoi, rulați codul care pune cap la cap întreg atacul implementat. Ciphertext-ul va fi un text "citibil".
Pentru a vă putea verifica, aveți cheia lăsată în schelet, dar nu veți primi punctajul pe acest Task dacă nu implemntați atacul diferențial.


In [ ]:
import random
from collections import Counter
from itertools import repeat


N = 64
SDIFF = "0" + "1" * (N - 2) + "0"
DIFF = int(SDIFF, 2)
assert DIFF == 2**63 - 2


def bin_to_bytes(bitstring: str) -> bytes:
    """Converteste string binar in bytes"""
    return int(bitstring, 2).to_bytes(len(bitstring) // 8, byteorder="big")


def complement(bitstring: str, *index: int) -> str:
    """
    Calculează complementul în binar pentru fiecare bit specificat in `index`,
    din stringul binar `bitstring`. Funcția returnează un nou string.

    Exemple utilizare:
    >>> complement("010110", 0)
    '110110'
    >>> complement("010110", -1)
    '010111'
    >>> complement("010110", 0, -1)
    '110111'
    >>> complement("010110", 0, -1, 2, 1, 0)
    '101111'
    """
    array = bytearray(bitstring, encoding="utf-8")
    for idx in index:
        array[idx] = ord("1") if bitstring[idx] == "0" else ord("0")
    return array.decode()


def extended_gcd(a: int, b: int) -> tuple[int, int, int]:
    """Algoritmul lui Euclid extins, pentru determinarea celui mai mare
    divizor comun, împreună cu coeficienții lui Bézout x, y care satisfac
    relația:
                            ax + by = gcd(a, b)

    Funcția returnează cel mai mare divizor comun și coeficienții x, y.
    A se vedea și https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm
    """
    x, y, u, v = 0, 1, 1, 0
    while a != 0:
        q, r = b // a, b % a
        m, n = x - u * q, y - v * q
        b, a, x, y, u, v = a, r, u, v, m, n
    gcd = b
    return gcd, x, y


def condition_1(
    ciphertexts: list[tuple[str, str]]
) -> tuple[list[tuple[str, str]], list[tuple[str, str]]]:
    """Funcția verifică ce ciphertext-uri rămân mai departe, conform
    condițiilor atacului din secțiunea 4 al articolului [1].

    [1] Vladimir Furman, Differential Cryptanalysis of Nimbus, 2002,
    https://link.springer.com/content/pdf/10.1007/3-540-45473-X_16.pdf
    """
    thirdLSBis1 = 0
    thirdLSBis0 = 0

    # Conditions:
    #  (1) The matching pairs must have the bits 10 as the two least significant
    #      bits of the ciphertext XOR difference.
    #  (2) The matching pairs must have 0 as the least significant bit of the
    #      ciphertexs.
    #  (3) All matching pairs must have the same third least significant bit of
    #      their ciphertext XOR difference. Note that we use this criterion
    #      when, there is a majority of matching pairs.
    precandidates = []
    for ctext in ciphertexts:
        ci0, ci1 = map(int, ctext, repeat(16))
        cxor = ci0 ^ ci1
        cb0 = f"{ci0:064b}"
        cb1 = f"{ci1:064b}"
        cbxor = f"{cxor:064b}"

        # Verifică condițiile (1) și (2)
        if cbxor[-2:] != "10" or cb0[-1] != "0" or cb1[-1] != "0":
            continue
        precandidates.append(ctext)

        # Numără pentru condiția de majoritate din (3)
        if cbxor[-3] == "1":
            thirdLSBis1 += 1
        else:
            thirdLSBis0 += 1

    # Definește majoritatea pentru (3)
    if thirdLSBis1 >= thirdLSBis0:
        defining3rdlsb = "1"
    else:
        defining3rdlsb = "0"

    # Selecteaza perechile care satisfac (3) drept candidat, și restul drept
    # non-candidat
    candidates = []
    noncandidates = []
    for pre in precandidates:
        ci0, ci1 = map(int, pre, repeat(16))
        cxor = ci0 ^ ci1
        cbxor = f"{cxor:064b}"
        if cbxor[-3] == defining3rdlsb:
            candidates.append(pre)
        else:
            noncandidates.append(pre)

    return candidates, noncandidates


# TODO: Implementați funcția care generează mesajele plaintext.
def create_plaintexts() -> list[tuple[str, str]]:
    """
    Funcția va genera in total 128 de perechi de mesaje random după urmatoarele
    reguli:
      I) Perechea [p1, p2]
        - Un plaintext p1 de dimensiune 62 biți, unde primul și ultimul bit
          vor avea valoarea 0, iar ceilalți biți vor fi random
        - Un mesaj p2 obținut prin p2 = p1 XOR diff
      II) Perechea [complement MSB p1, complement MSB p2]
      III) Perechea [complement MSB și LSB p1, complement MSB și LSB p2]
      IV) Perechea [complement LSB p1, complement LSB p2]

      Se vor genera 32 de stringuri random, restul fiind variații ale acestui
      mesaj conform regulilor I-IV.
    """
    plaintexts = []

    ########## SOLUȚIA AICI ##########

    for _ in range(32):
        # Regula I: p1 cu primul și ultimul bit = 0
        middle = ''.join(random.choice("01") for _ in range(62))
        p1 = '0' + middle + '0'  # 64 biți
        p2 = bitxor(p1, SDIFF)

        # Regula II: flip MSB (bit 0)
        p1_2 = complement(p1, 0)
        p2_2 = complement(p2, 0)

        # Regula III: flip MSB + LSB (bit 0 și 63)
        p1_3 = complement(p1, 0, 63)
        p2_3 = complement(p2, 0, 63)

        # Regula IV: flip LSB (bit 63)
        p1_4 = complement(p1, 63)
        p2_4 = complement(p2, 63)

        # Adaugă cele 4 perechi
        plaintexts.extend([
            (p1, p2),
            (p1_2, p2_2),
            (p1_3, p2_3),
            (p1_4, p2_4)
        ])
    assert len(plaintexts) == 128
    ##################################

    return plaintexts


# TODO: Generați toate ciphertext-urile corespunzătoare plaintext-urilor
# generate cu create_plaintexts(). Folosiți funcția bin_to_bytes() dacă
# trebuie să convertiți un string binar în bytes.
def get_ciphertexts(plaintexts: list[tuple[str, str]]) -> list[tuple[str, str]]:
    """Funcția cripteaza toate plaintext-urile date, folosind cifrul Nimbus.
    Returneaza o listă de perechi de ciphertexts, de aceeasi lungime cu
    argumentul `plaintexts`.
    """
    ciphertexts = []

    for pair in plaintexts:
        assert len(pair) == 2

        fin = []
        for plain in pair:
            assert len(plain) == 64

            ########## SOLUȚIA AICI ##########

            plain_bytes = bin_to_bytes(plain)
            ciphertext = encrypt_block(plain_bytes)

            ##################################

            fin.append(ciphertext)

        ciphertexts.append(tuple(fin))

    assert len(ciphertexts) == len(plaintexts)
    return ciphertexts


# TODO: Verificați ce ciphertext-uri respectă condițiile observate pentru
# anumiți biți. Numărați câte soluții ale ecuației impuse lui K_odd se repetă
# pentru aceste ciphertext-uri.
def get_candidate_subkeys(
    ciphertexts: list[tuple[str, str]]
) -> list[tuple[int, int]]:
    """Calculează candidații care satisfac Condiția 1 din [1], apoi pentru
    fiecare pereche candidat găsită, se rezolvă ecuația
                        DIFF * K' = C1 + C2 (mod 2^64)                      (1)

    Ecuația (1) este echivalentă cu:
      DIFF * K' = m * 2^64 + (C1 + C2) <=> DIFF * K' - m * 2^64 = C1 + C2   (2)

    Dacă C1 + C2 se divide la gcd(DIFF, 2^64) = gcd(2^64, DIFF) = d, atunci
    ecuația (1) admite d soluții [2] care pot fi determinate folosind algoritmul
    extins al lui Euclid. Cum DIFF = 2^63 - 2 = 2 * (2^62 - 1), rezultă că 2
    este un factor comun (2^62 - 1 și 2^63 sunt numere coprime). Impărțind la 2
    ecuația (2), putem calcula inversul multiplicativ lui DIFF / 2. Prin urmare,
    avem:
                DIFF / 2 * K' - m * 2^63 = (C1 + C2) / 2                    (3)

    care se poate rescrie:
                 DIFF / 2 * K' = (C1 + C2) / 2      (mod 2^63)              (4)

    cu soluția:
            K' = (C1 + C2) / gcd(DIFF, 2^64) * x     (mod 2^63)             (5)

    unde x este inversul multiplicativ al lui DIFF / 2 și îi corespunde
    coeficientul Bézout pentru DIFF. Din ecuația (5) se poate observa însă că
    K' este mărginit de 2^63. Din acest motiv, MSB este cel care ne determină
    cele 2 soluții din ecuația (1). Prin urmare, (1) admite drep soluții
    0 || K' și 1 || K'.

    Salvăm ambele soluții, întrucât ele vor fi filtrate ulterior.

    [1] Vladimir Furman, Differential Cryptanalysis of Nimbus, 2002,
    https://link.springer.com/content/pdf/10.1007/3-540-45473-X_16.pdf
    [2] https://en.wikipedia.org/wiki/Modular_multiplicative_inverse
    [3] https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm
    """
    counter = Counter()
    gcdd, x, _ = extended_gcd(DIFF, 1 << 64)
    candidates, _ = condition_1(ciphertexts)

    for cand in candidates:
        ci1, ci2 = map(int, cand, repeat(16))
        ########## SOLUȚIA AICI ##########

        # TODO: Asigurați-vă că 2 divide ci1 + ci2 și că suma ci1 + ci2 nu
        # este unul dintre DIFF și 2^64.
        total = ci1 + ci2
        if total % gcdd != 0 or total in (DIFF, 1 << 64):
            continue

        # TODO: Calculați k_odd_1 folosind x obținut prin gcd și relația:
        # K' = (C1 + C2) / gcd(DIFF, 2^64) * x     (mod 2^63)
        # Această soluție reprezintă 0 || K'
        diff_half = DIFF // 2
        mod_half = (1 << 64) // 2
        total_half = total // 2

        _, x_inv, _ = extended_gcd(diff_half, mod_half)
        x_inv %= mod_half

        k_odd_1 = (total_half * x_inv) % mod_half

        # TODO: Calculați cea de-a doua soluție 1 || K', prin modificarea MSB
        # din k_odd_1
        k_odd_2 = k_odd_1 | (1 << 63)

        assert mod_64(DIFF * k_odd_1) == mod_64(ci1 + ci2)
        assert mod_64(DIFF * k_odd_2) == mod_64(ci1 + ci2)

        # TODO: Actualizați counterul pentru k_odd_1 și k_odd_2
        counter[k_odd_1] += 1
        counter[k_odd_2] += 1

        ##################################

    return counter.most_common(1)


# TODO: Păstrați subcheile determinate folosind get_candidate_subkeys().
# În cazul în care nu există va trebui să se reîncerce alte subkei.
# parțiale. Dacă numărul de apariții este prea mic (<= 1), va trebui să încercăm
# alte subchei. În cazul celor două eșecuri, returnați None. De asemenea,
# returnați și cheile pare, întrucât nu știm dacă K era par sau impar (doar
# de K_odd știm ca era impar).
def find_subkey(ciphertexts: list[tuple[str, str]]) -> list[int] | None:
    """Returnează potențialele chei care urmează a fi verificate. Dacă nu
    există, returnează None."""
    ########## SOLUȚIA AICI ##########

    # TODO: Determinați subcheile candidat și verificați dacă avem unul. Dacă
    # nu, returnați None.
    best = get_candidate_subkeys(ciphertexts)
    if not best or len(best) == 0:
        return None

    # TODO: Extrageți cheia și numărul de apariții. Daca numărul de apariții e
    # prea mic, returnați None
    k_odd, count = best[0]
    if count <= 1:
        return None

    # TODO: Calculați complementul lui k_odd (flip MSB), fiind o soluție
    # potențială.
    k_odd_alt = k_odd ^ (1 << 63)

    # TODO: Întrucât nu știm dacă subcheia era pară sau impară, returnați ambele
    # variante. În total, funcția returnează 4 subchei:
    # k_odd, k_odd_alt, k_odd XOR 1, k_odd_alt XOR 1
    return [k_odd, k_odd_alt, k_odd ^ 1, k_odd_alt ^ 1]

    ##################################


# TODO: Implementare atac. Funcția care va genera toate cheile posibile,
# folosind o abordare DFS, în care pentru runda curentă se vor căuta canditații
# posibili pentru subcheia curentă.
def find_key(
    plaintexts: list[tuple[str, str]],
    ciphertexts: list[tuple[str, str]],
    key_candidates: list[int],
) -> list[int] | None:
    # Starea curentă este determinată de tuplul: (candidați cheie, ciphertexts
    # decriptate de rundă curentă, cheia curentă găsită)
    states = [(key_candidates, ciphertexts, [])]

    while len(states) > 0:
        key_candidates, ciphertexts, crt_key = states.pop()
        if len(crt_key) > ROUNDS:
            continue

        for k in key_candidates:
            dciphertexts: list[tuple[str, str]] = ciphertexts[:]
            ########## SOLUȚIA AICI ##########
            # TODO: Actualizați dciphertexts cu decriptarile fiecărui ciphertext
            # cu o rundă din Nimbus. Știți ca o rundă este de forma:
            #                 C_i+1 = K_odd * g(C_i XOR K)
            # Pentru decriptare, determinați cum puteți scoate C_i. Va trebui
            # să păstrați formatul hexadecimal.

            new_ciphertexts = []
            for c0_hex, c1_hex in dciphertexts:
                # Convertim hex → int
                c0 = int(c0_hex, 16)
                c1 = int(c1_hex, 16)

                # Inversem pașii: y = K_odd * g(x ^ K)
                # Deci: x = g⁻¹( y * K_odd⁻¹ mod 2^64 ) ^ K
                # K_odd = k, K = k (în atac presupunem aceiași valoare)
                inv_k = pow(k | 1, -1, 1 << 64)  # K_odd^-1 mod 2^64

                x0 = g(pow(c0, 1, 1 << 64) * inv_k % (1 << 64)) ^ k
                x1 = g(pow(c1, 1, 1 << 64) * inv_k % (1 << 64)) ^ k

                # Formatăm înapoi ca hex
                x0_hex = format(x0, "x").rjust(16, "0")
                x1_hex = format(x1, "x").rjust(16, "0")

                new_ciphertexts.append((x0_hex, x1_hex))

            # TODO: Ne oprim când cipertext-ul descrifrat este același cu
            # plaintext-ul. Condiția trebuie să fie adevărată pentru toate
            # perechile (plaintext, ciphertext). În acest caz, doar returnați
            # cheia găsită.
            if all(
                bin_to_bytes(p1) == bytes.fromhex(x0)
                and bin_to_bytes(p2) == bytes.fromhex(x1)
                for (p1, p2), (x0, x1) in zip(plaintexts, new_ciphertexts)
            ):
                return [k] + crt_key

            ##################################

            # Contruim trail-ul mai departe recursiv
            dkcandidates = find_subkey(new_ciphertexts)
            if dkcandidates is None:
                continue

            states.append((dkcandidates, new_ciphertexts, [k] + crt_key))

    return None


# Vom repeta atacul până vom găsi o cheie potrivită.
# Generarea fiind nedeterministă (random), posibil să fim nevoiți să
# reluăm atacul într-o buclă. Dacă scriptul durează foarte mult (mai mult de
# câteva secunde), foarte probabil ceva nu este corect implementat; foarte
# puțin probabil doar nu ați avut noroc să găsiți cheia.
def attack() -> list[int]:
    while True:
        plaintexts = create_plaintexts()
        assert len(plaintexts) == 128

        ciphertexts = get_ciphertexts(plaintexts)
        assert len(ciphertexts) == 128

        kcands = find_subkey(ciphertexts)
        if kcands is not None:
            key = find_key(plaintexts, ciphertexts, kcands)
            if key is not None:
                return key


# TODO: Inversăm întreg algoritmul, rundă cu rundă, folosind cheia găsită
def decrypt(ciphertext: str, subkeys: list[int]) -> str:
    result = ""

    ########## SOLUȚIA AICI ##########
    # Împărțim ciphertext-ul în blocuri de 16 caractere hex (64 biți)
    for i in range(0, len(ciphertext), 16):
        c_block = int(ciphertext[i:i+16], 16)

        # Aplicăm cele 5 runde în ordine inversă
        for round_key in reversed(subkeys):
            # Inversăm: y = k_odd * g(x ^ k) => x = g⁻¹(y * k_odd⁻¹) ^ k
            k_odd = round_key | 1  # forțăm impar (oricum era deja)
            k_inv = pow(k_odd, -1, 1 << 64)  # invers mod 2^64

            c_block = g((c_block * k_inv) % (1 << 64)) ^ round_key

        # Adăugăm blocul la rezultat (hex, 16 caractere)
        result += format(c_block, "x").rjust(16, "0")
    ##################################

    return result


# Rulăm atacul și decriptăm flag-ul
flag = "d6b012d1d56d0006670e7f847a4ebe14ad69e8c46b472f76"
print("Encrypted Flag:", flag)

subkeys = attack()
print(f"Subkeys:", subkeys)

fullkey = []
for k in subkeys:
    hexkey = hex(k)[2:]
    for hc in range(0, len(hexkey) - 1, 2):
        fullkey.append(chr(int(hexkey[hc : hc + 2], 16)))
print("Full key:", "".join(fullkey))

plaintext = bytearray.fromhex(decrypt(flag, subkeys)).decode().rstrip('\x00')
print("Plaintext:", plaintext)